In [1]:
import random
import os
import glob
import shutil

In [2]:
random_country_dir = random.choice(os.listdir("../country_data"))
random_country_dir

'SYR_Syrian Arab Republic'

In [3]:
figures = glob.glob(f"../country_data/{random_country_dir}/figures/*.png", recursive = True)
figures

['../country_data/SYR_Syrian Arab Republic/figures/SYR_BP_1.png',
 '../country_data/SYR_Syrian Arab Republic/figures/SYR_GCP_1.png',
 '../country_data/SYR_Syrian Arab Republic/figures/SYR_Minx_top20_subsectors.png',
 '../country_data/SYR_Syrian Arab Republic/figures/SYR_UNFCCC_NAI_1.png',
 '../country_data/SYR_Syrian Arab Republic/figures/SYR_CAIT_gases_1.png',
 '../country_data/SYR_Syrian Arab Republic/figures/SYR_CO2_totals.png',
 '../country_data/SYR_Syrian Arab Republic/figures/SYR_CDIAC_1.png',
 '../country_data/SYR_Syrian Arab Republic/figures/SYR_EIA_1.png',
 '../country_data/SYR_Syrian Arab Republic/figures/SYR_IEA_1.png']

In [4]:
random_fig = random.choice(figures)
random_fig

'../country_data/SYR_Syrian Arab Republic/figures/SYR_GCP_1.png'

In [5]:
shutil.copyfile(random_fig, "../supplemental/random_daily_fig.png",)

'../supplemental/random_daily_fig.png'

In [6]:
print("random_fig path imported")

random_fig path imported
